# Задание 1 

Кроулинг осуществлялся по поддомену Википедии en.wikipedia.org с помощью паука wiki_spider (файл /wiki/spiders/wiki_spider.py). При этом рассматривались лишь ссылки, которые находятся в основной части статьи и не являются фрагментными (ссылаются на ту же страницу), новыми (для них не существует страниц) или служебными. С каждой скачанной страницы извлекалось максимум 100 новых ссылок. Стартовые страницы: https://en.wikipedia.org/wiki/IP_address, https://en.wikipedia.org/wiki/Leaves_of_Grass, https://en.wikipedia.org/wiki/Overdraft, https://en.wikipedia.org/wiki/Dark_matter.

Кроулинг запускался в двух режимах: обхода в глубину (dfo - по умолчанию) и обхода в ширину (bfo - включается последними тремя настройками в /wiki/settings.py). Процесс останавливался при скачивании 12000 страниц (параметр CLOSESPIDER_ITEMCOUNT в /wiki/settings.py). Скачанные страницы с обнаруженными на них ссылками лежат в /dfo_wiki_pages.json и /bfo_wiki_pages.json.

# Задание 2 

Построим по полученным данным граф. Граф строим ориентированный, но без параллельных ребер. Это позволит нам позже сравнить результаты работы алгоритмов ранжирования PageRank и HITS, который в используемой реализации не умеет работать с мультиграфами. Отметим также, что в граф добавляются все страницы, в том числе и нескачанные (для них нам известен только URL): так мы покроем больше ссылок.

Графы построим для обоих обходов (dfo и bfo):

In [17]:
import json
from wiki.items import WikiItem
import networkx as nx


def as_wiki_item(dct):
    wi = WikiItem()
    wi['url'] = dct['url']
    wi['title'] = dct['title']
    wi['snippet'] = dct['snippet']
    wi['outlinks'] = dct['outlinks']
    return wi

def parse_data(filename):
    wiki_items = None
    url2item = dict()
    with open(filename) as fd:
        wiki_items = json.load(fd, object_hook=as_wiki_item)
        for wi in wiki_items:
            url2item[wi['url']] = wi
    #
    G = nx.DiGraph()# nx.MultiDiGraph()
    for wi in wiki_items:
        G.add_edges_from([(wi['url'], link) for link in wi['outlinks']])
    #
    return (wiki_items, url2item, G)
    
dfo_wiki_items, dfo_url2item, dfo_G = parse_data('dfo_wiki_pages.json')
bfo_wiki_items, bfo_url2item, bfo_G = parse_data('bfo_wiki_pages.json')

print "Graph info (dfo_G):\n{}".format(nx.info(dfo_G))
print "\nGraph info (bfo_G):\n{}".format(nx.info(bfo_G))

Graph info (dfo_G):
Name: 
Type: DiGraph
Number of nodes: 239656
Number of edges: 493986
Average in degree:   2.0612
Average out degree:   2.0612

Graph info (bfo_G):
Name: 
Type: DiGraph
Number of nodes: 211740
Number of edges: 678378
Average in degree:   3.2038
Average out degree:   3.2038


Большее количество ребер в графе, построенном по обходу в ширину, объясняется тем, что при таком обходе глубина кроулинга мала (для используемых параметров около ~2-3) и  в первую очередь скачиваются страницы тематически близкие между собой с большим числом перекрестных ссылок. 

# Задание 3, 4 

Запустим PageRank c damping параметром по умолчанию (=0.85) на построенных графах dfo_G и bfo_G:

In [18]:
def topk(url2rank, k=10):
    return sorted(url2rank.iteritems(), key = lambda e: e[1], reverse = True)[:k]

def output(url2rank, url2item):
    for url, rank in url2rank:
        title, snippet = '<title:blank>', '<snippet:blank>'
        if url in url2item:
            wi = url2item[url]
            title = wi['title']
            snippet = wi['snippet']
        print "{} ({})\n{}\n{}\n".format(title.encode("utf-8"), rank, url.encode("utf-8"), snippet.encode("utf-8"))

def page_rank(G, url2item, alpha=0.85):
    print "\n################# PAGE RANK (alpha={}) #################\n".format(alpha)
    pr = nx.pagerank_scipy(G, alpha)
    top10_pr = topk(pr, k=10)
    output(top10_pr, url2item)

print "################# dfo_G #################\n"
page_rank(dfo_G, dfo_url2item)
print "\n\n################# bfo_G #################\n"
page_rank(bfo_G, bfo_url2item)

################# dfo_G #################


################# PAGE RANK (alpha=0.85) #################

<title:blank> (0.000144171030686)
https://en.wikipedia.org/wiki/United_States
<snippet:blank>

<title:blank> (6.93441772288e-05)
https://en.wikipedia.org/wiki/United_Kingdom
<snippet:blank>

<title:blank> (6.13961708964e-05)
https://en.wikipedia.org/wiki/World_War_II
<snippet:blank>

<title:blank> (5.2384582435e-05)
https://en.wikipedia.org/wiki/Canada
<snippet:blank>

<title:blank> (4.83968430892e-05)
https://en.wikipedia.org/wiki/France
<snippet:blank>

<title:blank> (4.67867151968e-05)
https://en.wikipedia.org/wiki/New_York_City
<snippet:blank>

<title:blank> (4.15105908705e-05)
https://en.wikipedia.org/wiki/Germany
<snippet:blank>

<title:blank> (4.12966640661e-05)
https://en.wikipedia.org/wiki/Mathematics
<snippet:blank>

<title:blank> (3.53651885262e-05)
https://en.wikipedia.org/wiki/Australia
<snippet:blank>

<title:blank> (3.4802371419e-05)
https://en.wikipedia.org/wiki/Londo

Из результатов выше легко видеть, что для случая обхода в ширину некоторые статьи из топ-10 пересекаются по тематике со стартовыми страницами, чего нельзя сказать про обход в глубину. Малая глубина кроулинга и, как следствие, тесное тематическое единство страниц объясняет и тот факт, что все страницы из топ-10 оказались загруженными.  Между тем, для обхода в глубину удалось покрыть больше тематических областей, что лучше в смысле более точного описания веб-графа Википедии. 

Далее будем работать только с графом dfo_G, построенным по обходу в глубину. Рассмотрим как изменяется топ-10 в зависимости от величины damping параметра alpha равного 1.0, 0.95, 0.85, 0.5, 0.3, 0.0:

In [19]:
del bfo_wiki_items
del bfo_url2item
del bfo_G

wiki_items = dfo_wiki_items
url2item = dfo_url2item
G = dfo_G

for alpha in (1.0, 0.95, 0.85, 0.5, 0.3, 0.0):
    page_rank(G, url2item, alpha=alpha)


################# PAGE RANK (alpha=1.0) #################

<title:blank> (0.000168876627726)
https://en.wikipedia.org/wiki/United_States
<snippet:blank>

<title:blank> (8.08450354231e-05)
https://en.wikipedia.org/wiki/United_Kingdom
<snippet:blank>

<title:blank> (7.14944397379e-05)
https://en.wikipedia.org/wiki/World_War_II
<snippet:blank>

<title:blank> (6.08925709598e-05)
https://en.wikipedia.org/wiki/Canada
<snippet:blank>

<title:blank> (5.62011129059e-05)
https://en.wikipedia.org/wiki/France
<snippet:blank>

<title:blank> (5.43068447971e-05)
https://en.wikipedia.org/wiki/New_York_City
<snippet:blank>

<title:blank> (4.80996397074e-05)
https://en.wikipedia.org/wiki/Germany
<snippet:blank>

<title:blank> (4.7847961114e-05)
https://en.wikipedia.org/wiki/Mathematics
<snippet:blank>

<title:blank> (4.08697545965e-05)
https://en.wikipedia.org/wiki/Australia
<snippet:blank>

<title:blank> (4.02076168233e-05)
https://en.wikipedia.org/wiki/London
<snippet:blank>


################# PAGE 

Во-первых, топ одинаков для всех alpha не равных нулю. Во-вторых, по мере уменьшения величины damping параметра, распределение значений PageRank для вершин графа все больше приближается к равномерному. Если посмотреть на результаты выше, то в 0 мы как раз и получили равномерное распределение, где вероятность посетить конкретную страницу одинакова для всех страниц. Полученные результаты вполне согласуются с известной теорией.

# Задание 6 

Запустим на графе G алгоритм HITS и выделим топ-10 страниц исходя из их хабовости (HUBS), авторитетности (AUTHORITIES) и метрики усредненной по этим двум значениям (MEAN):

In [20]:
print "\n################# HITS (HUBS) #################\n"
h, a = nx.hits(G)
top10_h = topk(h, k=10)
output(top10_h, url2item)

print "\n################# HITS (AUTHORITIES) #################\n"
top10_a = topk(a, k=10)
output(top10_a, url2item)

print "\n################# HITS (MEAN) #################\n"
ha = dict()
for url in set(h.iterkeys()).intersection(set(a.iterkeys())):
    ha[url] = (h[url] + a[url]) / 2.0
top10_ha = topk(ha, k=10)
output(top10_ha, url2item)


################# HITS (HUBS) #################

Treaty of San Francisco (0.00124088093882)
https://en.wikipedia.org/wiki/Treaty_of_San_Francisco
Treaty of San Francisco (サンフランシスコ講和条約, San-Furanshisuko kōwa-Jōyaku?), Peace Treaty with Japan (日本国との平和条約, Nihon-koku tono Heiwa-Jōyaku) or commonly known as the Treaty of Peace with Japan, Peace Treaty of San Francisco, or San Francisco Peace Treaty), mo...

Ferndale, California (0.00104934477945)
https://en.wikipedia.org/wiki/Ferndale,_California
Ferndale is a city in Humboldt County, California, United States. Its population was 1,371 at the 2010 census, down from 1,382 at the 2000 census. The city contains dozens of well-preserved Victorian storefronts and homes. Ferndale is the northern gateway...

Prairie du Chien, Wisconsin (0.00104126652427)
https://en.wikipedia.org/wiki/Prairie_du_Chien,_Wisconsin
Prairie du Chien (/ˌprɛri du ˈʃiːn/) is a city in and the county seat of Crawford County, Wisconsin, United States. The population was 5,

По понятным причинам в топ по HUBS попали только скачанные страницы, причем полученный список ни в одной позиции  не пересекается с подсчитанным ранее топом по PageRank. Топ по Authorities практически совпал с топом по PageRank, так как построение обеих метрик связано с использованием сведений о степенях полузахода вершин веб-графа. Использование усредненной метрики MEAN показало результат близкий к тому, что был получен для Authorities.